<a href="https://colab.research.google.com/github/AbshkPskr/Zomato-Analysis/blob/master/z_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings("ignore")
import plotly.graph_objs as go
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/AbshkPskr/Zomato-Analysis/master/reviews.csv",
                 names = ['name','rating','date','cust_rating','review','sentiment']).drop_duplicates()

In [4]:
rest_names = df.groupby('name').count().sort_values('rating',ascending = False).head(20).index.to_list()
df = df[df.name.isin(rest_names)]
df['rating'] = df['rating'].astype('float')
df['date'] = pd.to_datetime(df['date'])
df['cust_rating'] = df['cust_rating'].astype('float')
df['review'] = df['review'].astype('string')
df['sentiment'] = df['sentiment'].astype('float').round(2)

#Emotion Detection (Plutchik, ekman, poms)

In [ ]:
sc = open("req.txt","w")
sc.write('''h5py==2.9.0
Keras==1.1.0
numpy==1.16.0
pandas==1.1.5
python-dateutil==2.8.0
pytz==2018.9
PyYAML==5.1
scipy==1.2.1
six==1.12.0
Theano==1.0.4''')
sc.close()

In [ ]:
!pip install -r req.txt -q

     |████████████████████████████████| 2.8MB 7.7MB/s 
     |████████████████████████████████| 153kB 28.9MB/s 
     |████████████████████████████████| 17.3MB 248kB/s 
     |████████████████████████████████| 235kB 57.8MB/s 
     |████████████████████████████████| 276kB 56.7MB/s 
     |████████████████████████████████| 24.8MB 1.2MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [ ]:
pip install --upgrade pandas

Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.5)


In [ ]:
# !pip freeze > req.txt
# !pip uninstall -r req.txt -y -q

In [ ]:
import os;
os.environ['KERAS_BACKEND'] = 'theano'
import html
import pickle
import re

import pandas as pd
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import sequence

import requests

class EmotionPredictor:
    def __init__(self, classification, setting, use_unison_model=True):
        """
        Args:
            classification (str): Either 'ekman', 'plutchik', 'poms'
                or 'unison'.
            setting (str): Either 'mc' or 'ml'.
            use_unison_model (bool): Whether to use unison model;
                else use single model.
        """
        if classification not in ['ekman', 'plutchik', 'poms', 'unison']:
            raise ValueError('Unknown emotion classification: {}'.format(
                classification))
        if setting not in ['mc', 'ml']:
            raise ValueError('Unknown setting: {}'.format(setting))

        self.classification = classification
        self.setting = setting
        self.use_unison_model = use_unison_model
        self.model = self._get_model()
        self.embeddings_model = self._get_embeddings_model()
        self.char_to_ind = self._get_char_mapping()
        self.class_values = self._get_class_values()
        self.max_len = self._get_max_sequence_length()
        self.name = classification +"-"+setting

    def _download_model(self,file_name):
        dir = "https://github.com/AbshkPskr/Emotion-recognition/blob/master/models/"
        r = requests.get(dir+ file_name+"?raw=true")
        with open(file_name, 'wb') as f:
            f.write(r.content)

    def _get_model(self):
        self._loaded_model_filename = '{}{}-{}.h5'.format(
            'unison-' if self.use_unison_model else '',
            self.classification,
            self.setting,
        )
        name = self._loaded_model_filename
        self._download_model(name)
        return load_model(self._loaded_model_filename)

    def _get_embeddings_model(self):
        last_layer_output = K.function([self.model.layers[0].input,
                                        K.learning_phase()],
                                       [self.model.layers[-3].output])
        return lambda x: last_layer_output([x, 0])[0]

    def _get_char_mapping(self):
        self._download_model('allowed-chars.pkl')
        with open('allowed-chars.pkl', 'rb') as f:
            return pickle.load(f)

    def _get_class_values(self):
        if self.classification == 'ekman':
            return ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
        elif self.classification == 'plutchik':
            return ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise',
                    'Trust', 'Anticipation']
        elif self.classification == 'poms':
            return ['Anger', 'Depression', 'Fatigue', 'Vigour', 'Tension',
                    'Confusion']

    def _get_max_sequence_length(self):
        if self.use_unison_model or self.classification == 'poms':
            return 143
        elif self.classification in ['ekman', 'plutchik']:
            return 141

    def predict_classes(self, tweets):
        indices = self._tweet_to_indices(tweets)
        predictions = self.model.predict(indices, verbose=False)

        df = pd.DataFrame({'Text': tweets})
        if self.setting == 'mc':
            df['Emotion'] = [self.class_values[i] for i in
                        predictions.argmax(axis=-1)]
        else:
            predictions[predictions >= 0.5] = 1
            predictions[predictions < 0.5] = 0
            for emotion, values in zip(self.class_values, predictions.T):
                df[emotion] = values
        return df

    def predict_probabilities(self, tweets):
        indices = self._tweet_to_indices(tweets)
        predictions = self.model.predict(indices, verbose=False)

        df = pd.DataFrame({'Tweet': tweets})
        for emotion, values in zip(self.class_values, predictions.T):
            df[emotion] = values
        return df

    def embed(self, tweets):
        indices = self._tweet_to_indices(tweets)
        embeddings = self.embeddings_model(indices)

        df = pd.DataFrame({'Tweet': tweets})
        for index, values in enumerate(embeddings.T, start=1):
            df['Dim{}'.format(index)] = values
        return df

    def embedd(self, tweets):
        """ Here only for backwards compatibility. """
        return self.embed(tweets)

    def _tweet_to_indices(self, tweets):
        indices = []
        for t in tweets:
            t = html.unescape(t)                            # unescape HTML
            t = re.sub(r"http\S+", "", t)                   # remove normal URLS
            t = re.sub(r"pic\.twitter\.com/\S+", "", t)     # remove pic.twitter.com URLS
            indices.append([self.char_to_ind[char] for char in t])
        return sequence.pad_sequences(indices, maxlen=self.max_len)

In [ ]:
models = []
models.append(EmotionPredictor(classification='plutchik', setting='mc', use_unison_model=False))
models.append(EmotionPredictor(classification='plutchik', setting='ml', use_unison_model=False))
models.append(EmotionPredictor(classification='ekman', setting='mc', use_unison_model=False))
models.append(EmotionPredictor(classification='ekman', setting='ml', use_unison_model=False))
models.append(EmotionPredictor(classification='poms', setting='mc', use_unison_model=False))
models.append(EmotionPredictor(classification='poms', setting='ml', use_unison_model=False))

ValueError: ignored

In [ ]:
for model in models:
    emotion = model.predict_classes(df['review'][0:10].to_list())
    emotion.to_csv("data-"+model.name+".csv")#, mode = 'a', header = False, index=False)

#Emotion Detection (text2Emotion)

In [ ]:
pip install text2emotion

In [ ]:
import text2emotion as te
from unidecode import unidecode
from textblob import TextBlob

def GetSentiment(Text):
    uni_text = unidecode(Text)
    analysis = TextBlob(uni_text)
    sentiment = analysis.sentiment.polarity
    return sentiment

for rev in df['review'][0:20].to_list():
    print(rev)
    print(te.get_emotion(rev))
    print(GetSentiment(rev), "------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

In [ ]:
pd.read_csv("out.csv")

#Emotion Detection (DeepEmoji)

In [5]:
!pip3 install torch==1.0.1 -f https://download.pytorch.org/whl/cpu/stable 
!git clone https://github.com/huggingface/torchMoji
import os
os.chdir('torchMoji')
# !pip3 install -e
#if you restart the package, the notebook risks to crash on a loop
#I did not restart and worked fine

Looking in links: https://download.pytorch.org/whl/cpu/stable
     |████████████████████████████████| 560.1MB 32kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.0.1 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
y
Cloning into 'torchMoji'...
remote: Enumerating objects: 143, done.
remote: Total 143 (delta 0), reused 0 (delta 0), pack-reused 143
Receiving objects: 100% (143/143), 2.41 MiB | 5.38 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [6]:
!pip install emoji
!pip install unidecode

     |████████████████████████████████| 133kB 8.9MB/s 
     |████████████████████████████████| 245kB 7.5MB/s 


In [7]:
!python3 scripts/download_weights.py

About to download the pretrained weights file from https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0#
The size of the file is roughly 85MB. Continue? [y/n]
y
Downloading...
Running system call: wget https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0# -O /content/torchMoji/model/pytorch_model.bin
--2021-02-08 18:18:43--  https://www.dropbox.com/s/q8lax9ary32c7t9/pytorch_model.bin?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/q8lax9ary32c7t9/pytorch_model.bin [following]
--2021-02-08 18:18:43--  https://www.dropbox.com/s/raw/q8lax9ary32c7t9/pytorch_model.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd2d6554069a5d02718b84dfc5e.dl.dropboxusercontent.com/cd/0/inline/BIkaEdIxnlHYgzr

In [21]:
import numpy as np
import emoji, json
from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
from torchmoji.sentence_tokenizer import SentenceTokenizer
from torchmoji.model_def import torchmoji_emojis
from unidecode import unidecode
  
EMOJIS = ":joy: :unamused: :weary: :sob: :heart_eyes: :pensive: :ok_hand: :blush: :heart: :smirk: :grin: :notes: :flushed: :100: :sleeping: :relieved: :relaxed: :raised_hands: :two_hearts: :expressionless: :sweat_smile: :pray: :confused: :kissing_heart: :heartbeat: :neutral_face: :information_desk_person: :disappointed: :see_no_evil: :tired_face: :v: :sunglasses: :rage: :thumbsup: :cry: :sleepy: :yum: :triumph: :hand: :mask: :clap: :eyes: :gun: :persevere: :smiling_imp: :sweat: :broken_heart: :yellow_heart: :musical_note: :speak_no_evil: :wink: :skull: :confounded: :smile: :stuck_out_tongue_winking_eye: :angry: :no_good: :muscle: :facepunch: :purple_heart: :sparkling_heart: :blue_heart: :grimacing: :sparkles:".split(' ')
# EMOJIS = ":disappointed: :disappointed: :disappointed: :smile: :disappointed: :thumbsup: :smile: :smile: :smile: :smile: :smile: :confused: :smile: :disappointed: :smile: :smile: :thumbsup: :smile: :disappointed: :confused: :disappointed: :confused: :smile: :smile: :disappointed: :thumbsup: :disappointed: :smile: :disappointed: :thumbsup: :smile: :angry: :thumbsup: :disappointed: :disappointed: :smile: :smile: :angry: :thumbsup: :thumbsup: :disappointed: :disappointed: :angry: :angry: :disappointed: :disappointed: :smile: :smile: :thumbsup: :smile: :confused: :angry: :smile: :thumbsup: :angry: :disappointed: :thumbsup: :disappointed: :smile: :smile: :smile: :thumbsup: :thumbsup: :smile:".split(' ')

model = torchmoji_emojis(PRETRAINED_PATH)

with open(VOCAB_PATH, 'r') as f:
  vocabulary = json.load(f)
st = SentenceTokenizer(vocabulary, 30)

def deepmojify(sentence,top_n = 5):
    print(sentence)
    if pd.isna(sentence): return ""
    sentence  = unidecode(sentence)
    tokenized, _ ,_ = st.tokenize_sentences([sentence])
    prob = model(tokenized)[0]
    ind = np.argpartition(prob, -top_n)[-top_n:]
    emoji_ids = ind[np.argsort(prob[ind])][::-1]
    emojis = map(lambda x: EMOJIS[x], emoji_ids)
    return emoji.emojize(f" {' '.join(emojis)}", use_aliases=True)


In [20]:
df.columns

Index(['name', 'rating', 'date', 'cust_rating', 'review', 'sentiment'], dtype='object')

In [ ]:
df['emotion'] = [deepmojify(rev, top_n = 1) for rev in df['review']]

In [23]:
df.to_csv('final.csv')

In [24]:
from google.colab import files
files.download('final.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from unidecode import unidecode
emo = []
for _ in df['review'].to_list():
    if pd.isna(_) == False:
        em = deepmojify(unidecode(_), top_n = 1)
        # if em == ' 😷' : print(_,em)
        emo.append(em)
        # print(deepmojify(unidecode(_), top_n = 1))

In [ ]:
from PIL import Image 
img = Image.open('aa.png') 
  
# Output Images 
img.show() 

#Dash

In [25]:
da = df.groupby('emotion').count()
da.name = (da.name - min(da.name))/(max(da.name) - min(da.name))
da

,name,rating,date,cust_rating,review,sentiment
emotion,,,,,,
,0.051777,543,543,543,0,543
☺,0.004968,53,53,53,53,53
✋,0.001051,12,12,12,12,12
✌,0.003344,36,36,36,36,36
✨,0.023214,244,244,244,244,244
...,...,...,...,...,...,...
🙅,0.007356,78,78,78,78,78
🙈,0.001720,19,19,19,19,19
🙊,0.000287,4,4,4,4,4


In [26]:
!pip install emoji -q

In [27]:
!pip install jupyter-dash -q
!pip install dash-dangerously-set-inner-html -q

     |████████████████████████████████| 81kB 4.6MB/s 
     |████████████████████████████████| 1.0MB 8.8MB/s 
     |████████████████████████████████| 3.5MB 22.2MB/s 
     |████████████████████████████████| 194kB 33.4MB/s 
     |████████████████████████████████| 1.8MB 55.1MB/s 
     |████████████████████████████████| 358kB 37.8MB/s 


In [28]:
stylecss = '''body {
    font-family: "Open Sans", sans-serif;
    background-color: #1e2130;
    margin: 0;
}'''

import os
if not os.path.exists('assets'):
    os.makedirs('assets')
sc = open("assets/style.css","w")
sc.write(stylecss)
sc.close()

In [29]:
df.emotion.unique()

array(['', ' 👍', ' 😕', ' 😡', ' 😋', ' 💓', ' 😷', ' 😅', ' 😊', ' 😌', ' 🎶',
       ' 😑', ' ☺', ' ✨', ' 😍', ' 😜', ' 👌', ' ❤', ' 😠', ' 😢', ' 💪', ' 😄',
       ' 🙅', ' 😉', ' 😐', ' 😈', ' 💛', ' 😳', ' 😎', ' 😞', ' 😬', ' 👏', ' 💙',
       ' 👀', ' 💁', ' 💔', ' 🙏', ' 😔', ' ✌', ' 🔫', ' 🎵', ' 😪', ' 💀', ' 💯',
       ' 💖', ' 😖', ' 🙌', ' 😂', ' 😓', ' 😒', ' 😣', ' 🙈', ' 😘', ' 😴', ' 😤',
       ' 💜', ' 👊', ' ✋', ' 😭', ' 😁', ' 🙊'], dtype=object)

In [30]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash
import dash_table.DataTable as DT
from  dash_dangerously_set_inner_html import DangerouslySetInnerHTML as DSIH
import json

rest_dict = {}
for drop in rest_names:
    rest =  df[df.name == drop].sort_values('date')#[-400:]
    rest['date'] = rest['date'].astype('string')
    rest = rest[pd.notna(rest['review'])]
    rest['smooth_sentiment'] = rest['sentiment'].rolling(int(len(rest)/30)).mean()
    rest['smooth_cust_rating'] = (rest['cust_rating']/10).rolling(int(len(rest)/40)).mean()
    rest = rest[pd.notna(rest['smooth_sentiment'])]
    rest = rest.set_index(i for i in range(len(rest)))
    rest_dict[drop] = rest

# external_css = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__)#,external_stylesheets=external_css)
    
app_styling = {
    'background': '#1e2130',
    'layout': '#161a28',
    'text':'#000000',
    'text-shadow':'2px 2px 5px #0A76BA',
    'box-shadow':'4px 4px 5px #0E1017',
}

#'border':'1px solid grey',
dropdown_label_style = {'width':'10%','height':'40px','line-height': '35px','float':'left','color':'white','font-size':'20px','text-shadow': app_styling['text-shadow'],'text-align':'center'}
dropdown_div_style = {'width':'85%','height':'40px','float':'right','color':'black'}
graph_style = {'width': '70%','float': 'left'}
figure_text_style={'text-shadow': app_styling['text-shadow'],'box-shadow': app_styling['box-shadow']}
head_style = {'font-size':'20px','text-align':'center','text-shadow': app_styling['text-shadow'],'margin':'5%'}
text_style = {'font-size':'50px','text-align':'center','text-shadow': app_styling['text-shadow']}
rating_div_style = {'width': '50%','float': 'left','color':'white','height':'120px','background-color':app_styling['layout'],'box-shadow': app_styling['box-shadow']}
Sentiment_div_style = {'width': '49%','float': 'right','color':'white','height':'120px','background-color':app_styling['layout'],'box-shadow': app_styling['box-shadow']}
review_div_style = {'height':'282.5px','overflow-y':'scroll','background-color':app_styling['layout'],'box-shadow': app_styling['box-shadow']}
review_text_style = {'width': '86%','font-size':'17px','text-align':'centre','color':'white','margin':'3% 5%'}
graph_hover_data_style = {'width': '29%','float': 'right'}
gap_div = html.Div(style={'height':'15px','clear':'both'})

app.layout = html.Div([
                       html.Div([
                                #  html.H1("Zomato Review Analysis",style={'text-align': 'center','color':'White','font-size':'50px','text-shadow': app_styling['text-shadow']}),
                                 html.Div([html.Div('Select Restaurant', style=dropdown_label_style),
                                           html.Div(dcc.Dropdown(id='restaurant-dropdown',
                                                                 options=[{'label': i, 'value': i} for i in rest_names],
                                                                 value='PCO'),style=dropdown_div_style)
                                           ],
                                          ),
                                 gap_div,
                                 html.Div(children=[
                                                    html.Div(children = [
                                                                         dcc.Graph(id='graph',style = figure_text_style),
                                                                         gap_div, 
                                                                         html.Div([
                                                                                   dcc.Graph(id='radar',style={'width' : '49%','float': 'left','text-shadow': app_styling['text-shadow'],'box-shadow': app_styling['box-shadow']}),
                                                                                   dcc.Graph(id='pie',style={'width' : '49%','float': 'right','text-shadow': app_styling['text-shadow'],'box-shadow': app_styling['box-shadow']}),
                                                                                   ],
                                                                                  ),
                                                                         ],
                                                             style=graph_style
                                                             ),
                                                    html.Div(children=[
                                                                       html.Div(children=[
                                                                                          html.Div([
                                                                                                    html.Div(children='Rating',style = head_style),
                                                                                                    html.Div(id='cust_rating',style=text_style)
                                                                                                    ],
                                                                                                   style=rating_div_style
                                                                                                   ),
                                                                                          html.Div([
                                                                                                    html.Div(children='Sentiment',style = head_style),
                                                                                                    html.Div(id='sentiment',style=text_style)
                                                                                                    ],
                                                                                                   style=Sentiment_div_style
                                                                                                   ),
                                                                                          ],
                                                                                # style = {'height' : '100px'}
                                                                                ),
                                                                       gap_div,
                                                                       html.Div([
                                                                                 html.Div(id = 'review',style=review_text_style),
                                                                                 ],
                                                                                style = review_div_style
                                                                                ),
                                                                       gap_div,
                                                                       html.Div(
                                                                           html.Img(src="",style={'height':'100%', 'width':'100%'}),
                                                                           style = review_div_style),
                                                                       ],
                                                             style = graph_hover_data_style
                                                             ),
                                                    ],
                                          ),
                                 gap_div,
                                 html.Div(id='table'),
                                 ],
                                style={'margin':'4% 3%'}
                                ),
                       ],
                      style = {'background-color':app_styling['background']}
                      )


@app.callback(
    Output('table', 'children'),
    Input("get", "height")
)
def debug(value):
    return value

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Output('radar', 'figure'),
    Output('pie', 'figure'),
    Input("restaurant-dropdown", "value")
)

def update_figure(drop):
    fig_font_size = 10

    rest_data = rest_dict[drop]

    fig = go.Figure()
    # fig.add_trace(go.Scatter(x=rest_data.index,
    #                          y=rest_data.sentiment,
    #                          mode='lines+markers',
    #                          name='confirmed',line=dict(color='yellow', width=2)))
    # fig.add_trace(go.Scatter(x=rest_data.index,
    #                          y=rest_data.cust_rating,
    #                          mode='lines+markers',
    #                          name='confirmed',line=dict(color='blue', width=2)))
    fig.add_trace(go.Scatter(x=rest_data.index,
                             y=rest_data.smooth_sentiment,
                             mode='lines+markers',
                             name='Sentiment',line=dict(color='green', width=2)))
    fig.add_trace(go.Scatter(x=rest_data.index,
                             y=rest_data.smooth_cust_rating,
                             mode='lines+markers',
                             name='Customer Rating',line=dict(color='red', width=2)))
    fig.update_layout(margin=dict(l=60,r=10,b=20,t=50,pad=0),
                      paper_bgcolor=app_styling['layout'], 
                      height= 350,
                      legend=dict(x=.01,y=.98),
                      title_text = 'Reviews',
                      font_size=fig_font_size,
                      xaxis_title="Number of reviews",
                      yaxis_title="Sentiment and rating with rolling mean",
                    #   gridcolor = 'Red',
                    #   clickmode='event+select',
                      xaxis = {'gridcolor':'grey'},
                      yaxis = {'gridcolor':'grey'},
                      plot_bgcolor=app_styling['layout'],
                      font = {'color':'white'},
                      hovermode='x'
                      )
    
    radar_data = rest_data.groupby('emotion').count()
    radar_data.name = (radar_data.name - min(radar_data.name))/(max(radar_data.name) - min(radar_data.name))
    radar = go.Figure()
    radar.add_trace(go.Scatterpolar(r=radar_data.name.to_list(),
                                    theta=radar_data.index.to_list(),
                                    mode='lines+markers',fill='toself',
                                    line=dict(width=2,color='#4E7094')))
    
    radar.update_layout(margin=dict(l=50,r=50,b=20,t=50,pad=0),
                    paper_bgcolor=app_styling['layout'],
                    height= 350,
                    # legend=dict(x=.01,y=.98),
                    title_text = 'Emotions',
                    font_size=fig_font_size,
                    xaxis_title="none",
                    yaxis_title="Emotional value",
                    plot_bgcolor=app_styling['layout'],
                    font = {'color':'white'},
                    polar = dict(angularaxis = dict(tickfont = dict(size = 25)),
                                 radialaxis=dict(visible = True,range = [0,1.2])),
                    )

   
    positive,negative,neutral = 0,0,0
    for senti in rest_data['sentiment']:
        if senti > 0:
            positive += 1
        if senti < 0:
            negative += 1
        else:
            neutral += 1
    pie = go.Figure()
    pie.add_trace(go.Pie(values=[positive,negative,neutral],
                 labels= ['Positive','Negative','Neutral'],
                 text=['Positive','Negative','Neutral'],
                 marker={'colors' :['#2EB848','#B82E2E','#2D35B4']},
                 hole = 0.4))
    pie.update_layout(margin=dict(l=20,r=20,b=20,t=50,pad=0),
                      paper_bgcolor=app_styling['layout'],
                      height= 350,
                    #   legend=dict(x=.01,y=.98),
                      title_text = 'Reviews',
                      font_size=fig_font_size,
                      xaxis_title="name",
                      yaxis_title="Number of reviews",
                      clickmode='event+select',
                      plot_bgcolor=app_styling['layout'],
                      font = {'color':'white'},
                      hovermode='x')
    

    return [fig,radar,pie]

@app.callback(
        Output('cust_rating', 'children'),
        Output('sentiment', 'children'),
        Output('review', 'children'),
        Input("restaurant-dropdown", "value"),
        Input("graph", "hoverData"),
)
def GetHoverData(drop,hoverData):
    rest_data = rest_dict[drop]
    X = 12
    if hoverData != None : X = int(hoverData['points'][0]['x'])
    rest_data = rest_data[rest_data.index == X]
    # table_data = rest_data.transpose().reset_index()
    # table = html.Table(children=[
    #                              html.Thead(html.Tr(children=[html.Th(col) for col in table_data.columns.values],
    #                                                 style={'color':app_styling['text']}
    #                                                 )
    #                                         ),
    #                              html.Tbody([html.Tr(children=[html.Td(table_data) for table_data in d],
    #                                                  style={'color':app_styling['text']}
    #                                                  )
    #                                          for d in table_data.values.tolist()])
    #                             ])

    review = rest_data['review']
    cust_rating = rest_data['cust_rating']
    sentiment = rest_data['sentiment']
    return [cust_rating,sentiment,review]


if __name__ == '__main__':
    app.run_server(debug=True,port = 8020)#mode = "inline")

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
rest_data.transpose().reset_index()

In [ ]:
rest_data = rest_dict['Local']
rest_data = rest_data[rest_data.index == 22]
rest_data.values.tolist()

In [ ]:
aa = df[df.name == 'The Potbelly']
aa.sort_values('date')

In [ ]:
rest = df[df.name == "The Potbelly"].sort_values('date')

In [ ]:
rest['smooth_sentiment'] = rest['sentiment'].rolling(int(len(rest)/5)).mean()

In [ ]:
rest.smooth_sentiment.isna().sum()

#Bokeh in Dash

In [ ]:
# bokeh basics
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Create a blank figure with labels
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

In [ ]:
# da = pd.DataFrame({'r':[1, 2, 3],'theta':['a', 'b', 'c']})
# radar = go.Figure()
# radar.add_trace(go.Scatterpolar(r=da['r'],theta=da['theta'],mode='lines',fill='toself'))
# radar.update_layout(height=500)

#Terminal

In [ ]:
!kill -9 -1

In [ ]:
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [ ]:
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });

In [ ]:
exit()

#HTML in Dash

In [ ]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html


class CustomDash(JupyterDash):
    def interpolate_index(self, **kwargs):
        # Inspect the arguments by printing them
        print(kwargs)
        return '''
        <!DOCTYPE html>
        <html>
            <head>
                <title>My App</title>
            </head>
            <body>
                <h1>Addition of two numbers</h1>
                <p id="demo2"></p>
                {app_entry}
                {config}
                {scripts}
                <script>
                var x=5;
                var y=6;
                var z;
                z=x+y;
                document.getElementById("demo2").innerHTML="Sum of 5 and 6 is "+ z;
                </script>
                {renderer}
                <div id="custom-footer">My custom footer</div>
            </body>
        </html>
        '''.format(
            app_entry=kwargs['app_entry'],
            config=kwargs['config'],
            scripts=kwargs['scripts'],
            renderer=kwargs['renderer'])

app = CustomDash()

app.layout = html.Div('Simple Dash App')

app.run_server(debug=True,port = 8060)


In [ ]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html


class CustomDash(JupyterDash):
    def interpolate_index(self, **kwargs):
        # Inspect the arguments by printing them
        print(kwargs)
        return '''
                <html>
                <head>
                <title>Simple Radar Chart</title>
                <link rel="stylesheet" href="style.css"/>
                <script src="https://mbostock.github.com/d3/d3.js?2.5.0"></script>
                <script src="radar.js"></script>
                </head><body><h1>Simple Radar Chart</h1>
                {app_entry}
                {config}
                {scripts}
                {renderer}
                <div id="viz">
                </div>
                <script>loadViz();</script>
                </body>
                </html>
        '''.format(
            app_entry=kwargs['app_entry'],
            config=kwargs['config'],
            scripts=kwargs['scripts'],
            renderer=kwargs['renderer'])

app = CustomDash()

app.layout = html.Div('Simpleasdfasdfasdf Dash App')
app.interpolate_index(app_entry=None,scripts=radarjs,config=None,renderer=None)


app.run_server(debug=True,port = 8060)


In [ ]:
stylecss = '''.axis {
    shape-rendering: crispEdges;
}

.axis line {
    stroke: #000000;
    stroke-width: 2px;
}

.axis .ticks line {
    stroke: #4F4F4F;
    stroke-width: 2px;
}

.axis .ticks line.minor{
    stroke: #CCCCCC;
    stroke-width:1px;'''

sc = open("style.css","w")
sc.write(radarjs)
sc.close()

In [ ]:
radarjs = '''var series, 
    hours,
    minVal,
    maxVal,
    w = 400,
    h = 400,
    vizPadding = {
        top: 10,
        right: 0,
        bottom: 15,
        left: 0
    },
    radius,
    radiusLength,
    ruleColor = "#CCC";

var loadViz = function(){
  loadData();
  buildBase();
  setScales();
  addAxes();
  draw();
};

var loadData = function(){
    var randomFromTo = function randomFromTo(from, to){
       return Math.floor(Math.random() * (to - from + 1) + from);
    };

    series = [
      [],
      []
    ];

    hours = [];

    for (i = 0; i < 24; i += 1) {
        series[0][i] = randomFromTo(0,20);
        series[1][i] = randomFromTo(5,15);
        hours[i] = i; //in case we want to do different formatting
    }

    mergedArr = series[0].concat(series[1]);

    minVal = d3.min(mergedArr);
    maxVal = d3.max(mergedArr);
    //give 25% of range as buffer to top
    maxVal = maxVal + ((maxVal - minVal) * 0.25);
    minVal = 0;

    //to complete the radial lines
    for (i = 0; i < series.length; i += 1) {
        series[i].push(series[i][0]);
    }
};

var buildBase = function(){
    var viz = d3.select("#viz")
        .append('svg:svg')
        .attr('width', w)
        .attr('height', h)
        .attr('class', 'vizSvg');

    viz.append("svg:rect")
        .attr('id', 'axis-separator')
        .attr('x', 0)
        .attr('y', 0)
        .attr('height', 0)
        .attr('width', 0)
        .attr('height', 0);
    
    vizBody = viz.append("svg:g")
        .attr('id', 'body');
};

setScales = function () {
  var heightCircleConstraint,
      widthCircleConstraint,
      circleConstraint,
      centerXPos,
      centerYPos;

  //need a circle so find constraining dimension
  heightCircleConstraint = h - vizPadding.top - vizPadding.bottom;
  widthCircleConstraint = w - vizPadding.left - vizPadding.right;
  circleConstraint = d3.min([
      heightCircleConstraint, widthCircleConstraint]);

  radius = d3.scale.linear().domain([minVal, maxVal])
      .range([0, (circleConstraint / 2)]);
  radiusLength = radius(maxVal);

  //attach everything to the group that is centered around middle
  centerXPos = widthCircleConstraint / 2 + vizPadding.left;
  centerYPos = heightCircleConstraint / 2 + vizPadding.top;

  vizBody.attr("transform",
      "translate(" + centerXPos + ", " + centerYPos + ")");
};

addAxes = function () {
  var radialTicks = radius.ticks(5),
      i,
      circleAxes,
      lineAxes;

  vizBody.selectAll('.circle-ticks').remove();
  vizBody.selectAll('.line-ticks').remove();

  circleAxes = vizBody.selectAll('.circle-ticks')
      .data(radialTicks)
      .enter().append('svg:g')
      .attr("class", "circle-ticks");

  circleAxes.append("svg:circle")
      .attr("r", function (d, i) {
          return radius(d);
      })
      .attr("class", "circle")
      .style("stroke", ruleColor)
      .style("fill", "none");

  circleAxes.append("svg:text")
      .attr("text-anchor", "middle")
      .attr("dy", function (d) {
          return -1 * radius(d);
      })
      .text(String);

  lineAxes = vizBody.selectAll('.line-ticks')
      .data(hours)
      .enter().append('svg:g')
      .attr("transform", function (d, i) {
          return "rotate(" + ((i / hours.length * 360) - 90) +
              ")translate(" + radius(maxVal) + ")";
      })
      .attr("class", "line-ticks");

  lineAxes.append('svg:line')
      .attr("x2", -1 * radius(maxVal))
      .style("stroke", ruleColor)
      .style("fill", "none");

  lineAxes.append('svg:text')
      .text(String)
      .attr("text-anchor", "middle")
      .attr("transform", function (d, i) {
          return (i / hours.length * 360) < 180 ? null : "rotate(180)";
      });
};

var draw = function () {
  var groups,
      lines,
      linesToUpdate;

  highlightedDotSize = 4;

  groups = vizBody.selectAll('.series')
      .data(series);
  groups.enter().append("svg:g")
      .attr('class', 'series')
      .style('fill', function (d, i) {
          if(i === 0){
            return "green";
          } else {
            return "blue";
          }
      })
      .style('stroke', function (d, i) {
          if(i === 0){
            return "green";
          } else {
            return "blue";
          }
      });
  groups.exit().remove();

  lines = groups.append('svg:path')
      .attr("class", "line")
      .attr("d", d3.svg.line.radial()
          .radius(function (d) {
              return 0;
          })
          .angle(function (d, i) {
              if (i === 24) {
                  i = 0;
              } //close the line
              return (i / 24) * 2 * Math.PI;
          }))
      .style("stroke-width", 3)
      .style("fill", "none");

  groups.selectAll(".curr-point")
      .data(function (d) {
          return [d[0]];
      })
      .enter().append("svg:circle")
      .attr("class", "curr-point")
      .attr("r", 0);

  groups.selectAll(".clicked-point")
      .data(function (d) {
          return [d[0]];
      })
      .enter().append("svg:circle")
      .attr('r', 0)
      .attr("class", "clicked-point");

  lines.attr("d", d3.svg.line.radial()
      .radius(function (d) {
          return radius(d);
      })
      .angle(function (d, i) {
          if (i === 24) {
              i = 0;
          } //close the line
          return (i / 24) * 2 * Math.PI;
      }));
};'''

rr = open("radar.js","w")
rr.write(radarjs)
rr.close()